In [2]:
import argparse
import glob
import multiprocessing as mp
import numpy as np
import os
import tempfile
import time
import warnings
import cv2
import tqdm

from RestructModelings.config import get_cfg
from RestructModelings.data.detection_utils import read_image
from RestructModelings.utils.logger import setup_logger

from predictor import VisualizationDemo

/home/ai_car/anaconda3/envs/Pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%run DemoVideo.py --config-file COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml --video-input "demo/xe hu 3.mp4" --confidence-threshold 0.6 --output video-output.mp4 --opts 'models/pytorch/carpath/model_001.pth'

[09/26 09:22:02 detectron2]: Arguments: Namespace(confidence_threshold=0.6, config_file='COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml', input=None, opts='models/pytorch/carpath/model_001.pth', output='video-output.mp4', video_input='demo/xe hu 3.mp4', webcam=False)
[09/26 09:22:14 fvcore.common.checkpoint]: [Checkpointer] Loading from models/pytorch/carpath/model_001.pth ...


Shape of roi_heads.box_predictor.bbox_pred.bias in checkpoint is torch.Size([76]), while shape of roi_heads.box_predictor.bbox_pred.bias in model is torch.Size([28]).
roi_heads.box_predictor.bbox_pred.bias will not be loaded. Please double check and see if this is desired.
Shape of roi_heads.box_predictor.bbox_pred.weight in checkpoint is torch.Size([76, 1024]), while shape of roi_heads.box_predictor.bbox_pred.weight in model is torch.Size([28, 1024]).
roi_heads.box_predictor.bbox_pred.weight will not be loaded. Please double check and see if this is desired.
Shape of roi_heads.box_predictor.cls_score.bias in checkpoint is torch.Size([20]), while shape of roi_heads.box_predictor.cls_score.bias in model is torch.Size([8]).
roi_heads.box_predictor.cls_score.bias will not be loaded. Please double check and see if this is desired.
Shape of roi_heads.box_predictor.cls_score.weight in checkpoint is torch.Size([20, 1024]), while shape of roi_heads.box_predictor.cls_score.weight in model is to

WARNING [09/26 09:22:20 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (76,) in the checkpoint but (28,) in the model! You might want to double check if this is expected.
WARNING [09/26 09:22:20 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (76, 1024) in the checkpoint but (28, 1024) in the model! You might want to double check if this is expected.
WARNING [09/26 09:22:20 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (20,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
WARNING [09/26 09:22:20 fvcore.common.checkpoint]: Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (20, 1024) in the checkpoint but (8, 1024) in the model! You mi

[ERROR:0@670.053] global /io/opencv/modules/videoio/src/cap_ffmpeg_impl.hpp (2767) open Could not find encoder for codec_id=27, error: Encoder not found
[ERROR:0@670.053] global /io/opencv/modules/videoio/src/cap_ffmpeg_impl.hpp (2839) open VIDEOIO/FFMPEG: Failed to initialize VideoWriter
[ERROR:0@670.067] global /io/opencv/modules/videoio/src/cap.cpp (595) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.5.5) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /tmp/video_format_testq05g1fvd/test_file.mkv in function 'icvExtractPattern'


  0%|          | 0/1004 [00:00<?, ?it/s]/home/ai_car/anaconda3/envs/Pytorch/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ig

TypeError: float() argument must be a string or a number, not 'Image'

In [1]:
data = {'CoordinateXY': [[318.4377136230469,
   266.90216064453125,
   793.2587280273438,
   373.11688232421875],
  [217.21653747558594,
   162.1292266845703,
   541.9356079101562,
   270.7200622558594],
  [373.2832336425781, 345.8374938964844, 799.4790649414062, 617.3776245117188],
  [176.8825225830078,
   225.94271850585938,
   225.0623321533203,
   286.9627685546875],
  [76.73395538330078,
   165.83758544921875,
   167.88363647460938,
   403.0719909667969],
  [45.843074798583984,
   330.0364074707031,
   121.32776641845703,
   445.63751220703125],
  [129.9160614013672,
   142.7761688232422,
   253.61709594726562,
   466.1328430175781],
  [264.994140625, 409.9341735839844, 411.2208557128906, 623.3927001953125]],
 'PartsCar': [13, 8, 7, 14, 3, 18, 9, 18]}

In [2]:
data

{'CoordinateXY': [[318.4377136230469,
   266.90216064453125,
   793.2587280273438,
   373.11688232421875],
  [217.21653747558594,
   162.1292266845703,
   541.9356079101562,
   270.7200622558594],
  [373.2832336425781, 345.8374938964844, 799.4790649414062, 617.3776245117188],
  [176.8825225830078,
   225.94271850585938,
   225.0623321533203,
   286.9627685546875],
  [76.73395538330078,
   165.83758544921875,
   167.88363647460938,
   403.0719909667969],
  [45.843074798583984,
   330.0364074707031,
   121.32776641845703,
   445.63751220703125],
  [129.9160614013672,
   142.7761688232422,
   253.61709594726562,
   466.1328430175781],
  [264.994140625, 409.9341735839844, 411.2208557128906, 623.3927001953125]],
 'PartsCar': [13, 8, 7, 14, 3, 18, 9, 18]}

In [7]:
listxmin, listxmax, listymin, listymax = [], [], [], []


In [1]:
from tensorflow.keras.models import load_model

model = load_model('models/carColorClassify-car color-85.60.h5')
model.summary()

2022-09-17 14:21:27.289686: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-17 14:21:27.315792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-17 14:21:27.316273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2022-09-17 14:21:27.316295: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-09-17 14:21:27.320799: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-09-17 14:21:27.320843: I tensorflow/stream_executor/platform/default/d

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [2]:
model = load_model('models/carColorClassify.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [1]:
from rembg import remove
from PIL import Image
input_path = '/home/ai_car/dev/inputDemo/white-range-rover-evoque-austria-alps-latest-brand-new-white-range-rover-evoque-beautiful-car-suv-nature-deep-alps-range-115177893.jpg'
output_path = '/home/ai_car/dev/outputDemo/white.png'
inputs = Image.open(input_path)
output = remove(inputs)
output.save(output_path)

In [2]:
import os
arr = os.listdir('/home/ai_car/dev/inputDemo')

In [6]:
for i in range(len(arr)):
    try:
        inputs = Image.open('/home/ai_car/dev/inputDemo/' + arr[i])
        output_path = '/home/ai_car/dev/outputDemo/' + arr[i]
        output = remove(inputs)
        output.save(output_path)
    except:
        pass

In [5]:
f = open("requirements.txt", "r")
list_library = list(f.read().split('\n'))
for i in range(len(list_library)):
    component = list_library[i].split(' ')
    list_library[i] = component[0]
    

In [6]:
with open(r'requirements2.txt', 'w') as fp:
    for item in list_library:
        fp.write("%s\n" % item)
        print('done')

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
